In [13]:
import json
import numpy as np
import pyperclip
from scipy.stats import ttest_ind_from_stats, ttest_rel, wilcoxon,ttest_ind,ttest_rel
import os
os.chdir(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
from FIBI.data.analyse_cpp_results.clustering.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class
import pandas as pd
from scipy.stats import ranksums, mannwhitneyu
import plotly.express as px
import bs4
from itertools import product
# https://www.youtube.com/watch?v=CIbJSX-biu0 for ressources

In [22]:
LnPts = ['20','50','100']
Lassign = ['RAND','IMPR']
threshold_n = 1000

## Investigate Wilcoxon computation

In [15]:
from h5py import File
data_rd = []
with File("data/data_algo_same/table1_exp_clust.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data_rd.append(dico_value)
df_rd = pd.DataFrame(data_rd)
data_impr = []
with File("data/data_algo_same/table1_exp_clust_impr.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data_impr.append(dico_value)
df_impr = pd.DataFrame(data_impr)
df_impr['INIT'] = 'IMPR'
df_rd['INIT'] = 'RAND'

df = pd.concat([df_rd,df_impr])


In [20]:
df_1 = df.copy()
df_1.loc[:,"ratio"] = df_1.loc[:,"final_cost"]/df_1.loc[:,"init_cost"]
df_1.sort_values(by=['SEED_POINTS','SEED_ASSIGN'])
backup_stats = {}
backup_soups = {}
for init in Lassign:
    backup_stats[init] = {}
    backup_soups[init] = {}
    dico_df = {}
    for nPts in LnPts:
        backup_stats[init][nPts] = {}
        backup_soups[init][nPts] = {}
        dico_df[nPts] = {}
        for nClust in df_1['NUM_CLUST'].unique():
            if int(nClust) >= int(nPts):
                continue
            backup_stats[init][nPts][nClust] = {}
            backup_soups[init][nPts][nClust] = {}
            soup = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style.css"></head><body></body></html>""")
            dico_df[nPts][nClust] = {}
            for algo in ['FICL','BICL']:
                dico_df[nPts][nClust][algo] = df_1.query(f'INIT=="{init}" & NUM_POINTS == {nPts} & IMPR_CLASS == "{algo}" & NUM_CLUST == {nClust}')
                dico_df[nPts][nClust][algo].sort_values(by=['SEED_POINTS','SEED_ASSIGN'])
            soup.html.body.append(bs4.BeautifulSoup(f"<h2>{nPts} points, initialisation {init}, {nClust} clusters</h2>"))
            table = soup.new_tag(f"table")
            table.attrs['id'] = "data"
            table.attrs['cellspacing'] = "0"
            table.attrs['cellpadding'] = "0"
            L = ["coutFinalFICL/coutInitial","coutFinalBICL/coutInitial"]
            for col in [*L,L[0]+'-'+L[1]]:
                colTag = soup.new_tag('th')
                colTag.string = col
                table.append(colTag)
            FICL = np.array(dico_df[nPts][nClust]['FICL']['final_cost']/dico_df[nPts][nClust]['FICL']['init_cost'])
            BICL = np.array(dico_df[nPts][nClust]['BICL']['final_cost']/dico_df[nPts][nClust]['BICL']['init_cost'])
            if len(FICL) != len(BICL):
                raise Exception(f'diff size: {len(FICL)}, {len(BICL)}')
            FICL_BICL = FICL-BICL
            for records in zip(FICL,BICL,FICL_BICL):
                tr = soup.new_tag('tr')
                for r in records:
                    td = soup.new_tag('td')
                    td.string = f"{r}"
                    tr.append(td)
                table.append(tr)
            soup.html.body.append(table)
            
            df_stats = pd.DataFrame(FICL_BICL,columns=['difference'])
            descr = df_stats.describe().applymap(lambda x: f"{x:0.2e}").reset_index().to_dict(orient='records')
            backup_stats[init][nPts][nClust] = descr
            backup_soups[init][nPts][nClust] = soup
for nClust in df_1['NUM_CLUST'].unique():
    div_stats = bs4.BeautifulSoup('<div id="stats"></div>')
    tmpLPts = [e for e in LnPts if int(e) > nClust]
    if len(tmpLPts) ==0:
        continue
    for init in Lassign:
        table = bs4.BeautifulSoup('<table></table>')
        header = bs4.BeautifulSoup(f'<tr><th colspan="{len(tmpLPts)+1}">{init}</th></tr><tr><th></th>'+''.join([f'<th>{n}</th>' for n in tmpLPts])+'</tr>')
        table.table.attrs['cellspacing'] = "0"
        table.table.attrs['cellpadding'] = "0"
        table.table.append(header)
        f = lambda L: [e['difference'] for e in L]
        stats_v = [f(L) for L in [backup_stats[init][k][nClust] for k in tmpLPts]]
        for stat,vals in zip([e['index'] for e in backup_stats[init][tmpLPts[0]][nClust]], zip(*stats_v)):
            tr = bs4.BeautifulSoup(f'<tr><th>{stat}</th></tr>')
            for v in vals:
                td = bs4.BeautifulSoup(f'<td>{v}</td>')
                tr.tr.append(td)
            table.table.append(tr)
        div_stats.div.append(table)

    for init in Lassign:
        for nPts in tmpLPts:
            backup_soups[init][nPts][nClust].body.append(bs4.BeautifulSoup(str(div_stats)))
            with open(f'FIBI/data/ttest/out_clust_{nClust}_clust_{nPts}pts_init_{init}.html','w') as f:
                f.write(str(backup_soups[init][nPts][nClust]))

## Compute Wilcoxon test pvalues

In [21]:
drop = ['key_dataset','num_iter','num_iter_glob','duration']
df_2 = df.copy()
df_2 = df_2[[c for c in df_2.columns if c not in drop]]
nPoints = [int(e) for e in df_2['NUM_POINTS'].unique() if int(e) <= threshold_n]
nPoints.sort()
soup = bs4.BeautifulSoup(f'<html><head><link rel="stylesheet" href="style2.css"></head><body></body></html>')
LnClust = df_2['NUM_CLUST'].unique()
LnClust.sort()
for nClust in LnClust:
    table = bs4.BeautifulSoup(f'<h2>P-values for {nClust} clusters</h2><table cellspacing="0" cellpadding="0"></table>')
    table.table.append(bs4.BeautifulSoup(f'<tr><th colspan="2"></th><th colspan="{len(nPoints)}">Number of points</th>'+'</tr>'))
    table.table.append(bs4.BeautifulSoup('<tr><th>INIT</th><th>Values provided to the test</th>'+''.join([f'<th>{n}</th>' for n in nPoints])+'</tr>'))
    for init in (Lassign):
        dico_stats = {'wilcoxon_direct':{},'wilcoxon_ratio':{}}
        for i,n in enumerate(nPoints):
            if n<=nClust:
                dico_stats['wilcoxon_direct'][n] = ""
                dico_stats['wilcoxon_ratio'][n] = ""
                continue
            tmp = df_2.query(f'INIT == "{init}" & NUM_POINTS == {n} & IT_ORDER=="CURR" & NUM_CLUST == {nClust}')
            tmpBI = tmp.query('IMPR_CLASS=="BICL"')
            tmpBI.sort_values(by=['SEED_POINTS','SEED_ASSIGN'])
            tmpFI = tmp.query('IMPR_CLASS=="FICL"')
            tmpFI.sort_values(by=['SEED_POINTS','SEED_ASSIGN'])
            res_wil = wilcoxon(tmpBI['final_cost'],tmpFI['final_cost'])
            res_ratio = wilcoxon(np.array(tmpBI['final_cost'])/np.array(tmpBI['init_cost']),np.array(tmpFI['final_cost'])/np.array(tmpFI['init_cost']))
            dico_stats['wilcoxon_direct'][n] = res_wil
            dico_stats['wilcoxon_ratio'][n] = res_ratio
        for i,(dic_id,name) in enumerate([['direct','Final cost'],['ratio','Final cost/Init cost']]):
            header = ""
            if i==0:
                header = f'<th rowspan="2">{init}</th>'
            tr = bs4.BeautifulSoup(f'<tr>{header}<th class="testUsed">{name}</th></tr>')
            for n,v in (dico_stats['wilcoxon_'+dic_id].items()):
                if isinstance(v,str):
                    tr.tr.append(bs4.BeautifulSoup(f"<td></td>"))
                else:
                    tr.tr.append(bs4.BeautifulSoup(f"<td class='{'more' if v.pvalue > 5e-2 else 'less'}'>{v.pvalue:.2e}</td>"))
            table.table.append(tr)
    soup.html.body.append(table)

with open(f'FIBI/data/ttest/out_clust_pvalues.html','w') as f:
    f.write(str(soup))
